In [ ]:
!pip install segmentation-models
!pip install keras.utils

In [71]:
# Importing libraries for the implementation
import os
import cv2
from tensorflow.python.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import numpy as np
from keras.utils import generic_utils
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
import segmentation_models as sm
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# For accessing files in drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
# File location in drive
file_root = '/content/drive/MyDrive/Colab Notebooks/Honours/unet/Semantic segmentation dataset/'

# Array to store images and masks of those images
imagearray=[]
maskarray=[]

# Adding images in imagearray list
for path,subs,files in os.walk(file_root):
    dirname=path.split(os.path.sep)[-1]
    if dirname=="images":
        images=os.listdir(path)
        for i,name in enumerate(images):
            if name.endswith(".jpg"):
                image=cv2.imread(path+"/"+name,1)
                image=cv2.resize(image,(256,256))
                image=np.array(image)
                imagearray.append(image)

# Adding masks in maskarray list
for path,subs,files in os.walk(file_root):
    dirname=path.split(os.path.sep)[-1]
    if dirname=="masks":
        masks=os.listdir(path)
        for i,M_name in enumerate(masks):
            if M_name.endswith(".png"):
                mask =cv2.imread(path+"/"+M_name,1)
                mask =cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)
                mask =cv2.resize(mask,(256,256))
                mask =np.array(mask)
                maskarray.append(mask)

# Converting imagearray list and maskarray list to array
imagedata = np.array(imagearray)
maskdata =  np.array(maskarray)

In [77]:
# Colors of various classes in the model
Building=np.array([60,16,152])
Land=np.array([132,41,246])
road=np.array([110,193,228])
Vegetation=np.array([254,221,58])
water=np.array([226,169,41])
Unlabeled =np.array([155,155,155])

# Storing maskdata in a variable
label = maskdata

# Function to generate flat labels for the given label
def flatLabels(label):
    label_seg = np.zeros(label.shape, dtype=np.uint8)
    label_seg[np.all(label == Building, axis=-1)] = 0
    label_seg[np.all(label == Land, axis=-1)] = 1
    label_seg[np.all(label == road, axis=-1)] = 2
    label_seg[np.all(label == Vegetation, axis=-1)] = 3
    label_seg[np.all(label == water, axis=-1)] = 4
    label_seg[np.all(label == Unlabeled, axis=-1)] = 5

    label_seg = label_seg[:, :, 0]  # Just take the first channel, no need for all 3 channels

    return label_seg

labels = []

# Converting maskdata to flat label and storing it in labels list
for i in range(maskdata.shape[0]):
    label = flatLabels(maskdata[i])
    labels.append(label)

# Converting labels list to array
labels = np.array(labels)
labels = np.expand_dims(labels, axis=3)

In [ ]:
# Selecting and displaying random image from dataset and its mask
import random
image_number = random.randint(0, len(imagearray))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(imagearray[image_number])
plt.subplot(122)
plt.imshow(labels[image_number][:,:,0])
plt.show()

In [ ]:
# Finding number of unique classes from labels array
n_classes = len(np.unique(labels))

# Seperating all the unique labels
labels_cat = to_categorical(labels, num_classes=n_classes)

# Splitting data in 8:2 ratio for training and testing purpose
X_train, X_test, y_train, y_test = train_test_split(imagedata, labels_cat, test_size = 0.20)

# Weights are optional if not defined it will automatically be set to 1
weights = [0.155, 0.155, 0.155, 0.155, 0.155, 0.155]

# Getting dice loss
dice_loss=sm.losses.DiceLoss(class_weights=weights)
focal=sm.losses.CategoricalFocalLoss()
total=dice_loss+(1*focal)

In [ ]:
# Function to find jacard_coef for loss
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

# Function to build unet model
def unet(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
    # Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    # Define input layer
    s = inputs

    # Contraction part
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    # Define output layer
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    # Creating the model
    model = Model(inputs=[inputs], outputs=[outputs])
    
    # Returning the model
    return model

In [ ]:
# Training the model on 30 epochs
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

metrics=['accuracy', jacard_coef]

def get_model():
    return unet(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

model = get_model()
model.compile(optimizer='adam', loss=total, metrics=metrics)
model.summary()

history1 = model.fit(X_train, y_train,
                    batch_size = 16,
                    verbose=1,
                    epochs=30,
                    validation_data=(X_test, y_test),
                    shuffle=False)

In [ ]:
# Plotting Training and validation loss
history = history1
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotting Training and validation IoU
acc = history.history['jacard_coef']
val_acc = history.history['val_jacard_coef']
plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()

In [ ]:
# Testing the model trained through a randomly selected image from test dataset
x_test_argmax=np.argmax(X_test, axis=3)

test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=x_test_argmax[test_img_number]

test_img_input=np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]

In [ ]:
# Plotting the predicted segmentation image
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img)
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth)
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img)
plt.show()